In [4]:
!pip install geneticalgorithm

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neural_network import MLPClassifier
#import skfuzzy as fuzz
from sklearn.base import BaseEstimator, ClassifierMixin
from geneticalgorithm import geneticalgorithm as ga



In [6]:
data=pd.read_csv('/content/postpartum_data.csv')

In [7]:
data

,Age,FamilySupport,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,EPDS_Score,RiskLevel
0,27,Medium,3,3,0,1,1,0,1,2,2,2,15,Profound
1,20,Medium,0,0,0,2,3,3,2,1,0,2,13,Severe
2,25,Low,3,2,1,3,0,2,2,2,3,2,20,Profound
3,25,High,2,0,0,0,2,1,2,2,2,2,13,Severe
4,18,Medium,1,2,0,2,3,2,0,3,3,0,16,Profound
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,28,Low,3,0,1,3,1,2,2,0,2,1,15,Profound
2996,30,Low,3,1,3,3,3,0,3,0,2,0,18,Profound
2997,39,Low,0,1,0,0,1,1,3,2,0,1,9,Mild
2998,39,High,0,2,1,1,3,2,1,3,2,1,16,Profound


In [8]:
le=LabelEncoder()
y=le.fit_transform(data['RiskLevel'])

In [9]:
y

array([2, 3, 2, ..., 0, 2, 2])

In [10]:
# Feature matrix and target
X = data.drop(columns=['RiskLevel'])

In [11]:
X

,Age,FamilySupport,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,EPDS_Score
0,27,Medium,3,3,0,1,1,0,1,2,2,2,15
1,20,Medium,0,0,0,2,3,3,2,1,0,2,13
2,25,Low,3,2,1,3,0,2,2,2,3,2,20
3,25,High,2,0,0,0,2,1,2,2,2,2,13
4,18,Medium,1,2,0,2,3,2,0,3,3,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,28,Low,3,0,1,3,1,2,2,0,2,1,15
2996,30,Low,3,1,3,3,3,0,3,0,2,0,18
2997,39,Low,0,1,0,0,1,1,3,2,0,1,9
2998,39,High,0,2,1,1,3,2,1,3,2,1,16


In [13]:
from sklearn.compose import ColumnTransformer
# 3. Identify column types **before splitting**
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# 4. Split data (X_train remains a DataFrame)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Build preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numerical_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])



In [15]:
from sklearn.pipeline import Pipeline
models = {
    "SVC": SVC(),
    "RandomForest": RandomForestClassifier(random_state=42),
    "FFNN": MLPClassifier(hidden_layer_sizes=(16, 8), max_iter=500, random_state=42)
}

# 8. Train and evaluate each model
for name, model in models.items():
    print(f"\n===== {name} =====")
    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("classifier", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


===== SVC =====
Accuracy: 0.975
Report:
               precision    recall  f1-score   support

        Mild       0.97      0.76      0.85        38
    Moderate       0.92      0.98      0.95       111
    Profound       0.99      1.00      1.00       332
      Severe       0.98      0.97      0.98       119

    accuracy                           0.97       600
   macro avg       0.96      0.93      0.94       600
weighted avg       0.98      0.97      0.97       600


===== RandomForest =====
Accuracy: 0.995
Report:
               precision    recall  f1-score   support

        Mild       1.00      0.92      0.96        38
    Moderate       0.97      1.00      0.99       111
    Profound       1.00      1.00      1.00       332
      Severe       1.00      1.00      1.00       119

    accuracy                           0.99       600
   macro avg       0.99      0.98      0.99       600
weighted avg       1.00      0.99      0.99       600


===== FFNN =====
Accuracy: 1.0
Repor

In [16]:
# 6. Full pipeline
pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", MLPClassifier(hidden_layer_sizes=(16, 8), max_iter=500, random_state=42))
])

# 7. Train model
pipeline.fit(X_train, y_train)

# 8. Predict and evaluate
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [17]:
import joblib

joblib.dump(le, "label_encoder.pkl")
joblib.dump(pipeline, "ppd_model_pipeline.pkl")

['ppd_model_pipeline.pkl']

In [ ]:
'''cols = ['Age', 'Q1', 'Q2','Q3','Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10','EPDS_Score' ,'FamilySupport']
X_test1 = [[25, 3, 3, 0, 1, 1, 0, 1, 2, 2, 2, 15, 'Medium']]
X_test1_df = pd.DataFrame(X_test1, columns=cols)

model.predict(X_test1_df)'''